# Manipulação de dados com Pandas II

## Agregando Dados

Vamos utilizar o mesmo dataset da aula anterior de introdução ao Pandas, analisando também os preços dos produtos.

In [1]:
import pandas as pd


In [2]:
df = pd.read_csv('orders.csv')
df

,id,first_name,last_name,email,shoe_type,shoe_material,shoe_color,price
0,41874,Kyle,Peck,KylePeck71@gmail.com,ballet flats,faux-leather,black,385.0
1,31349,Elizabeth,Velazquez,EVelazquez1971@gmail.com,boots,fabric,brown,388.0
2,43416,Keith,Saunders,KS4047@gmail.com,sandals,leather,navy,346.0
3,56054,Ryan,Sweeney,RyanSweeney14@outlook.com,sandals,fabric,brown,344.0
4,77402,Donna,Blankenship,DB3807@gmail.com,stilettos,fabric,brown,289.0
...,...,...,...,...,...,...,...,...
94,21506,Scott,Deleon,Scott.Deleon@gmail.com,stilettos,fabric,black,374.0
95,77266,Zachary,Gregory,Zachary.Gregory@gmail.com,sandals,leather,red,216.0
96,67264,Danielle,Merrill,DMerrill1998@gmail.com,wedges,faux-leather,red,461.0
97,19100,Danielle,Barron,DBarron1982@gmail.com,sandals,leather,white,313.0


São funções de agregação básicas: contagem, valores máximo e mínimo, média, desvio padrão e variância dos valores.

In [3]:
print("count: ", df.price.count())
print("max: ", df['price'].max())
print("min: ", df['price'].min())
print("média: ", df['price'].mean())
print("std: ", df['price'].std())
print("var: ", df['price'].var())


count:  98
max:  493.0
min:  91.0
média:  289.1326530612245
std:  110.95477278241789
var:  12310.961603197982


São estatísticas básicas a mediana (2º quartil) e quartis:

In [4]:
print("mediana: ", df['price'].median())
print("quantile: ", df['price'].quantile(0))
print("quantile: ", df['price'].quantile(0.25))
print("quantile: ", df['price'].quantile(0.5))
print("quantile: ", df['price'].quantile(0.75))
print("quantile: ", df['price'].quantile(1))

mediana:  285.5
quantile:  91.0
quantile:  205.5
quantile:  285.5
quantile:  373.5
quantile:  493.0


In [6]:
describe = df.describe().round(2)
describe['price']

count     98.00
mean     289.13
std      110.95
min       91.00
25%      205.50
50%      285.50
75%      373.50
max      493.00
Name: price, dtype: float64

Para as colunas que não possuem valores numéricos, é possível estabelecer algumas verificações, como quantos e quais valores únicos cada uma delas possuem:

In [7]:
print(df.shoe_material.unique())
print(df.shoe_material.nunique())

['faux-leather' 'fabric' 'leather']
3


Para agrupar informações importantes, como, por exemplo, o preço mais alto por tipo de calçado, podemos utilizar a função `groupby`:

In [8]:
pricey_shoes = df.groupby('shoe_type').price.max()
pricey_shoes

shoe_type
ballet flats    481.0
boots           478.0
clogs           493.0
sandals         456.0
stilettos       487.0
wedges          461.0
Name: price, dtype: float64

Para transformar o resultado do agrupamento em um novo dataframe, basta acrescentar `reset_index()` no fim:

In [9]:
pricey_shoes = df.groupby('shoe_type').price.max().reset_index()
pricey_shoes

,shoe_type,price
0,ballet flats,481.0
1,boots,478.0
2,clogs,493.0
3,sandals,456.0
4,stilettos,487.0
5,wedges,461.0


E assim podemos renomear as colunas, se for necessário:

In [10]:
pricey_shoes = pricey_shoes.rename(columns={'price': 'max_price'})
pricey_shoes

,shoe_type,max_price
0,ballet flats,481.0
1,boots,478.0
2,clogs,493.0
3,sandals,456.0
4,stilettos,487.0
5,wedges,461.0


Para realizar o agrupamento com mais de uma coluna, basta passar uma lista como parâmetro da função `groupby`:

In [11]:
pricey_shoes_2 = df.groupby(['shoe_type', 'shoe_color']).price.max().reset_index().rename(columns={'price': 'max_price'})

pricey_shoes_2

,shoe_type,shoe_color,max_price
0,ballet flats,black,385.0
1,ballet flats,brown,481.0
2,ballet flats,red,331.0
3,ballet flats,white,308.0
4,boots,black,171.0
5,boots,brown,388.0
6,boots,navy,478.0
7,boots,red,435.0
8,boots,white,339.0
9,clogs,black,451.0


In [13]:
shoe_counts = df.groupby(['shoe_type', 'shoe_color']).id.count().reset_index()

shoe_counts = shoe_counts.rename(columns={'id':'count'})

shoe_counts

,shoe_type,shoe_color,count
0,ballet flats,black,2
1,ballet flats,brown,5
2,ballet flats,red,3
3,ballet flats,white,5
4,boots,black,3
5,boots,brown,5
6,boots,navy,6
7,boots,red,2
8,boots,white,3
9,clogs,black,4


Para melhor compreensão, analise outro exemplo: A Biblioteca Municipal tem várias filiais espalhadas pela cidade. Em cada uma delas são  coletados todos os dados de retirada de livros em um DataFrame chamado `checkouts`. 

O DataFrame contém as colunas `location`, `date` e `book_title`. Se quisermos comparar o número total de livros retirados em cada filial, qual código poderíamos usar?

```
checkouts.groupby(['location']).book_title.count().reset_index()
``` 

No exemplo anterior, vimos que muitos dos dados se repetiram. Isso acontece porque fizemos o agrupamento por mais de uma coluna, o que pode ser resolvido estabelecendo um tabela pivô:

```
df.pivot(columns='ColumnToPivot',
         index='ColumnToBeRows',
         values='ColumnToBeValues')
```

In [14]:
shoe_max_pivot = pricey_shoes_2.pivot(columns='shoe_color', index='shoe_type', values='max_price').reset_index()

shoe_max_pivot

shoe_color,shoe_type,black,brown,navy,red,white
0,ballet flats,385.0,481.0,NaN,331.0,308.0
1,boots,171.0,388.0,478.0,435.0,339.0
2,clogs,451.0,323.0,493.0,439.0,324.0
3,sandals,NaN,344.0,346.0,409.0,456.0
4,stilettos,451.0,481.0,487.0,474.0,342.0
5,wedges,289.0,429.0,406.0,461.0,180.0


In [15]:
shoe_counts_pivot = shoe_counts.pivot(columns='shoe_color', index='shoe_type', values='count').reset_index()

shoe_counts_pivot

shoe_color,shoe_type,black,brown,navy,red,white
0,ballet flats,2.0,5.0,NaN,3.0,5.0
1,boots,3.0,5.0,6.0,2.0,3.0
2,clogs,4.0,6.0,1.0,4.0,1.0
3,sandals,1.0,4.0,5.0,3.0,4.0
4,stilettos,5.0,3.0,2.0,2.0,2.0
5,wedges,3.0,4.0,4.0,5.0,2.0


Ou aplicar duas ou mais métricas usando a função `agg`:

In [16]:
max_min_shoes = df.groupby('shoe_type').price.agg(['max', 'min'])
max_min_shoes

,max,min
shoe_type,,
ballet flats,481.0,91.0
boots,478.0,100.0
clogs,493.0,93.0
sandals,456.0,113.0
stilettos,487.0,167.0
wedges,461.0,115.0


É possível mesclar o cálculo de estatísticas com o agrupamento dos dados. Por exemplo, para calcular o percentil de 25% (1º quartil) do preço com base no agrupamento por cor do calçado, teremos os sapatos mais baratos assim:

In [17]:
import numpy as np

cheap_shoes = df.groupby('shoe_color').price.apply(lambda x: np.percentile(x, 25)).reset_index()

cheap_shoes

,shoe_color,price
0,black,NaN
1,brown,193.5
2,navy,205.5
3,red,250.0
4,white,196.0


## Ordenando Dados

In [20]:
df.sort_values(['price', 'shoe_type'], ascending=False)

,id,first_name,last_name,email,shoe_type,shoe_material,shoe_color,price
64,13553,Aaron,Hanson,AH3867@gmail.com,clogs,faux-leather,navy,493.0
38,87313,Wayne,Small,WSmall1987@gmail.com,stilettos,faux-leather,navy,487.0
34,85959,Julie,Rosa,JRosa1990@outlook.com,stilettos,leather,brown,481.0
18,35693,Margaret,Tyler,MTyler1974@gmail.com,ballet flats,fabric,brown,481.0
39,18507,Donna,Clay,DC4155@hotmail.com,boots,fabric,navy,478.0
...,...,...,...,...,...,...,...,...
36,24421,Megan,Rhodes,MR1115@gmail.com,boots,fabric,black,100.0
84,67253,Amber,Daugherty,AmberDaugherty60@gmail.com,clogs,faux-leather,red,95.0
86,39447,Denise,Mendoza,DM2227@outlook.com,clogs,faux-leather,brown,93.0
12,39587,Dennis,Vega,Dennis.Vega@gmail.com,ballet flats,faux-leather,brown,91.0


## Concatenando Dados

In [21]:
tabela_1 = pd.read_csv('tabela1.csv')
tabela_1

,nome,telefone,carros
0,João,121212,azul
1,João,343434,preto
2,Pedro,565656,verde
3,Caio,787878,amarelo


In [22]:
tabela_2 = pd.read_csv('tabela2.csv')
tabela_2

,nome,irmãos
0,João,1
1,Marcelo,3
2,Thiago,2
3,Caio,2


In [23]:
tabela_3 = pd.read_csv('tabela3.csv')
tabela_3

,nome,telefone,carros
0,João,121212,BRANCO
1,João,343434,PRATA
2,Pedro,565656,PRATA
3,Caio,787878,VERMELHO


Veremos como este método se comporta em dois cenários:

### Cenário 1

Vejamos como a método se comporta no caso de tabelas com as mesmas colunas. O método gera uma tabela com todas as combinações de colunas.

In [39]:
pd.concat([tabela_1, tabela_3], ignore_index=True)

,nome,telefone,carros
0,João,121212,azul
1,João,343434,preto
2,Pedro,565656,verde
3,Caio,787878,amarelo
4,João,121212,BRANCO
5,João,343434,PRATA
6,Pedro,565656,PRATA
7,Caio,787878,VERMELHO


### Cenário 2

Importaremos uma nova tabela chamada tabela_4, idêntica à tabela_3, porém com uma coluna a mais, a coluna ‘Animais’ :

In [26]:
tabela_4 = pd.read_csv('tabela4.csv')
tabela_4

,nome,telefone,carros,animais
0,João,121212,BRANCO,cachorro
1,João,343434,PRATA,gato
2,Pedro,565656,PRATA,cachorro


In [27]:
t = pd.concat([tabela_1, tabela_4]).reset_index(drop=True)
t

,nome,telefone,carros,animais
0,João,121212,azul,NaN
1,João,343434,preto,NaN
2,Pedro,565656,verde,NaN
3,Caio,787878,amarelo,NaN
4,João,121212,BRANCO,cachorro
5,João,343434,PRATA,gato
6,Pedro,565656,PRATA,cachorro


Como podemos ver, as linhas que vieram da tabela_1 receberam ‘NaN’ na coluna ‘Animais’. Previsivelmente, afinal essa coluna só existe na tabela_4.

Aqui, cada linha isolada não faz sentido. Mas o usuário pode filtrar a tabela para obter a informação que deseja.

O código abaixo, por exemplo, nos retorna todos ‘Animais’ relacionados a ‘Nome’ igual a ‘João’, excluindo aqueles valores que aparecem como NaN:

In [28]:
t[(t['nome'] == 'João') & (t['animais'].notna())]['animais']

4    cachorro
5        gato
Name: animais, dtype: object

Para ajustar os índices, basta incluir o parâmetro `ignore_index`:

In [29]:
t = pd.concat([tabela_1, tabela_4], ignore_index=True)
t

,nome,telefone,carros,animais
0,João,121212,azul,NaN
1,João,343434,preto,NaN
2,Pedro,565656,verde,NaN
3,Caio,787878,amarelo,NaN
4,João,121212,BRANCO,cachorro
5,João,343434,PRATA,gato
6,Pedro,565656,PRATA,cachorro


Para concatenar horizontalmente, basta incluir o parâmetro `axis=1`:

In [30]:
t = pd.concat([tabela_1, tabela_4], axis=1)
t

,nome,telefone,carros,nome,telefone,carros,animais
0,João,121212,azul,João,121212.0,BRANCO,cachorro
1,João,343434,preto,João,343434.0,PRATA,gato
2,Pedro,565656,verde,Pedro,565656.0,PRATA,cachorro
3,Caio,787878,amarelo,NaN,NaN,NaN,NaN


## Mesclando Dados

Caso queiramos a interseção exata entre as tabelas: `how = 'inner'`. Obtivemos a interseção entre as duas tabelas. Somente os valores na coluna “Nome” que existem em ambas tabelas aparecem no nosso resultado.

In [31]:
inner = pd.merge(tabela_1, tabela_2, how='inner', on='nome')
inner

,nome,telefone,carros,irmãos
0,João,121212,azul,1
1,João,343434,preto,1
2,Caio,787878,amarelo,2


Caso queiramos todas as informações, de ambas tabelas, fazemos um merge `how = 'outer'`. Perceba que "Pedro" não possui dados para “Irmãos”. E “Marcelo” e “Thiago” não possuem dados para “Telefone” e “Carros”. Volte nas tabelas 1 e 2 veja que esse é exatamente o resultado esperado se quisermos todas as informações.

In [32]:
outer = pd.merge(tabela_1, tabela_2, how='outer', on='nome')
outer

,nome,telefone,carros,irmãos
0,Caio,787878.0,amarelo,2.0
1,João,121212.0,azul,1.0
2,João,343434.0,preto,1.0
3,Marcelo,NaN,NaN,3.0
4,Pedro,565656.0,verde,NaN
5,Thiago,NaN,NaN,2.0


Um merge “left” ou “right” depende de qual tabela você deixa na direita ou esquerda. Para o seguinte cenário faremos um merge do tipo “left”. Mas o mesmo resultado pode ser obtido com um merge “right” trocando a posição das tabelas no método “merge”.

In [33]:
left = pd.merge(tabela_1, tabela_2, how='left', on='nome')
left

,nome,telefone,carros,irmãos
0,João,121212,azul,1.0
1,João,343434,preto,1.0
2,Pedro,565656,verde,NaN
3,Caio,787878,amarelo,2.0


In [34]:
right = pd.merge(tabela_2, tabela_1, how='right', on='nome')
right

,nome,irmãos,telefone,carros
0,João,1.0,121212,azul
1,João,1.0,343434,preto
2,Pedro,NaN,565656,verde
3,Caio,2.0,787878,amarelo


### Cenários para merge left

#### Cenário 1

Suponhamos que para a sua tarefa você deva manter os dados da tabela_1 e ir acrescentando colunas conforme mais dados sobre ‘João’, ‘Pedro’ e ‘Caio’ apareçam. Em outras palavras, somente ‘Nomes’ que existam na tabela_1 serão trazidos da tabela_2. Neste exemplo deixaremos a tabela_1 à esquerda e faremos um merge ‘left’:

In [35]:
m = pd.merge(tabela_1, tabela_2, how = 'left', on = 'nome')
m

,nome,telefone,carros,irmãos
0,João,121212,azul,1.0
1,João,343434,preto,1.0
2,Pedro,565656,verde,NaN
3,Caio,787878,amarelo,2.0


#### Cenário 2

Neste cenário uniremos a tabela_1 à tabela_3. Perceba que a nova tabela, tabela_3, é igual a tabela_1 mas com valores novos para a coluna ‘Carros’.

O que você acha que vai acontecer se tentarmos unir tabela_1 e tabela_3? Com qual valor para ‘Carros’ devemos ficar ao efetuarmos o seguinte merge?

In [36]:
m = pd.merge(tabela_1, tabela_3, how = 'left', on = 'nome')
m

,nome,telefone_x,carros_x,telefone_y,carros_y
0,João,121212,azul,121212,BRANCO
1,João,121212,azul,343434,PRATA
2,João,343434,preto,121212,BRANCO
3,João,343434,preto,343434,PRATA
4,Pedro,565656,verde,565656,PRATA
5,Caio,787878,amarelo,787878,VERMELHO


A nossa chave para o merge sendo ‘nome’, todas outras colunas iguais entre as tabelas são separadas em _x e _y, onde:

_x Corresponde aos valores que existiam na tabela da esquerda (tabela_1).
_y Corresponde aos valores que existiam na tabela da direita (tabela_3).

Esses sufixos podem ser alterados.

In [37]:
m = pd.merge(tabela_1, tabela_3, how = 'left', on = 'nome').rename(columns = {'carros_x': 'carro antigo', 'carros_y': 'carro novo', 'telefone_x': 'telefone primário', 'telefone_y': 'telefone secundário'})
m

,nome,telefone primário,carro antigo,telefone secundário,carro novo
0,João,121212,azul,121212,BRANCO
1,João,121212,azul,343434,PRATA
2,João,343434,preto,121212,BRANCO
3,João,343434,preto,343434,PRATA
4,Pedro,565656,verde,565656,PRATA
5,Caio,787878,amarelo,787878,VERMELHO


## Exercícios

In [54]:
# Aplique as quatro formas de join estudadas nos seguintes dataframes: 

df1 = pd.DataFrame({'A': [4, 2, 1, 3], 'B': ['a', 'b', 'c', 'b']})
df2 = pd.DataFrame({'A': [2, 3, 5], 'C': [10, 20, 30]})

df1


,A,B
0,4,a
1,2,b
2,1,c
3,3,b


In [55]:
df2

,A,C
0,2,10
1,3,20
2,5,30


In [61]:
left = pd.merge(df1, df2, how='left', on='A')
right = pd.merge(df1, df2, how='right', on='A')
inner = pd.merge(df1, df2, how='inner', on='A')
outer = pd.merge(df1, df2, how='outer', on='A')

outer

,A,B,C
0,1,c,NaN
1,2,b,10.0
2,3,b,20.0
3,4,a,NaN
4,5,NaN,30.0


In [63]:
# Sobre o resultado do outer join anterior, ordene a saída pela coluna A (ajustar índices). 

outer_ordered = outer.sort_values(['A'])
outer_ordered


,A,B,C
0,1,c,NaN
1,2,b,10.0
2,3,b,20.0
3,4,a,NaN
4,5,NaN,30.0


In [66]:
# Concatenar df1 e df2 em formato de linhas e depois colunas (ajustar índices).

df1_df2_line = pd.concat([df1, df2], ignore_index=True)
df1_df2_column = pd.concat([df1, df2], axis=1, ignore_index=True)

df1_df2_column

,0,1,2,3
0,4,a,2.0,10.0
1,2,b,3.0,20.0
2,1,c,5.0,30.0
3,3,b,NaN,NaN


In [67]:
# Calcule a soma da coluna A por agrupamento da coluna B no df1
df1_grouped_by_sum = df1.groupby(['B'])['A'].sum().reset_index().rename(columns={'A':'sum_A'})
df1_grouped_by_sum

,B,sum_A
0,a,4
1,b,5
2,c,1
